In [1]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.8
    Uninstalling idna-3.8:
      Successfully uninstalled idna-3.8


In [5]:
import cv2
from roboflow import Roboflow

# Initialize Roboflow with your API key
rf = Roboflow(api_key="lXZdRDe1SPy3KUdnj2G3")
project = rf.workspace().project("cc-tv-footage-annotation-b8-mx25p")
model = project.version(4).model

# Open the video file
video_path = '/content/2.mp4'
video_capture = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('save_output_video.avi', fourcc, 20.0, (int(video_capture.get(3)), int(video_capture.get(4))))

while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Run inference using numpy array
    prediction = model.predict(frame, confidence=50, overlap=30)

    # Draw predictions on the frame
    for item in prediction.json()['predictions']:
        x_center = int(item['x'])
        y_center = int(item['y'])
        width = int(item['width'])
        height = int(item['height'])
        class_label = item['class']
        confidence = item['confidence']

        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{class_label} {confidence:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write the frame with annotations
    out.write(frame)

video_capture.release()
out.release()
cv2.destroyAllWindows()

loading Roboflow workspace...
loading Roboflow project...
